## Part 1 - Closest to (0,0,0) in the long run

In [ ]:
import numpy as np
def load(file_name):
    '''Loads positions, velocities, and accelerations (as tuples of ints) from the file_name'''
    file = open(file_name, 'r')
    poss = []
    vels = []
    accs = []
    for s in file:
        poss.append(tuple(int(n) for n in s[s.find('p=<')+3:s.find('>, v=<')].split(',')))
        vels.append(tuple(int(n) for n in s[s.find('v=<')+3:s.find('>, a=<')].split(',')))
        accs.append(tuple(int(n) for n in s[s.find('a=<')+3:s.rfind('>')].split(',')))
    return np.array(poss), np.array(vels), np.array(accs)

In [ ]:
poss, vels, accs = load('day20.txt')
acc_mds = np.abs(accs).sum(axis=-1)  # Absolute values of accelerations of each particle
min_abs_acc = np.where(acc_mds == acc_mds.min())[0]  # Array of indexes with absolute acceleration = min
# If there were more than 1 such particles - initial velocity should have been considered.
print('Answer to part 1 - particle whose acceleration is the lowest = {}'.format(min_abs_acc))

## Part 2 - How many particles collided?

In [ ]:
p, v, a = np.array(poss), np.array(vels), np.array(accs)
print('Starting similation with {} particles'.format(len(p)))
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt
for t in range(100):
    #print('t = {}, {}, {}, {}'.format(t, p[-1], v[-1], a[-1]))
    unique, counts = np.unique(p, axis=0, return_counts=True)
    collision_mask = counts > 1
    if True in collision_mask:  # Collisions (non-unique positions) found
        collision_poss = unique[collision_mask]  # Duplicated positions (collisions) to be removed
        collision_idx = [i for i in range(len(p)) if p[i] in collision_poss]
        #print('----------\nt = {}, particles count = {}, collisions: {}, collision positions:\n{}'.format(
            #t, len(p), len(collision_idx), p[collision_idx]))
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d') 
        ax.scatter(p[:,0], p[:,1], p[:,2], s=1, c="g", marker="o")
        ax.scatter(collision_poss[:,0], collision_poss[:,1], collision_poss[:,2], s=50, c="r", marker="o")
        #ax.view_init(azim=180, elev=10)
        plt.show()

        p = np.delete(p, collision_idx, axis=0)
        v = np.delete(v, collision_idx, axis=0)
        a = np.delete(a, collision_idx, axis=0)
        print('t = {}, after collisions remain {} particles'.format(t, len(p)))
    v += a
    p += v
print('----------\nPart 2 answer: {}'.format(len(p)))

### Wrong answers
Too high: `772`.

Incorrect: `644`, `663`.